# Introduction

- This notebook is a cookbook for how to interact with an LLM via the OpenAI `ChatCompletion` API. Note that there are other cloud LLM providers that accept this API; here we use OpenAI for our backend.

- You must set your API key in the environment from which your iPython kernel was launched in order for `os.getenv()` to find it. You may not have an Org ID, that's not a problem.

# Setup

## Imports

In [ ]:
import os
import string
import pickle
import openai
from IPython.display import clear_output

## API keys

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_ID") # if you don't have an org id, just comment this line out

## Settings

In [ ]:
model = "gpt-4"
temperature = 1.0
out_dir_name = "output"

## Preamble

The 'system prompt' provides instructions to the LLM on how to behave. It should be specified as a dictionary inside a single-element list, e.g.,

`[{'role': 'system', 'content': 'You are BaseballGPT, an irritable knower of all things baseball.'}]`

Only a single system prompt can be provided.

In [ ]:
sys_prompt = []

The 'example interactions' provide the best form of instruction to the LLM, namely examples of how you would like it to behave. It should be specified as a list of dictionaries, e.g.,

`[{'role': 'user', 'content': 'Who was Babe Ruth?'}, {'role': 'assistant', 'content': 'An incredible baseball player. Or maybe you're talking about the chocolate bar?'}]`

You can add as many interactions as you'd like, provided you don't run out of 'context' for your prompt later.

In [ ]:
example_interactions = []

## Set up memory

'Memory' in this case is a running log of 'who said what' in the conversation

In [ ]:
memory = []
memory += sys_prompt
memory += example_interactions

# Utility functions

In [ ]:
def pretty_parse(memory_: list)-> str:
    """Parse the ChatCompletion API output and return it in a usefully formatted way.
    
        params:
            memory_: the `memory` list of dicts
            
        returns:
            a formatted string"""

    out_string = ""
    for turn in memory_:
        out_string += f"{turn['role'].capitalize()}: {turn['content']}\n"
    return out_string

In [ ]:
def prompt_LLM(memory_: list) -> str:
    """Send the running memory (which includes an unanswered prompt) to the LLM via an API call and return the response.
    
        params:
            memory_: the `memory` list of dicts
            
        returns:
            the text of the LLM response"""

    raw_response_ = openai.ChatCompletion.create(
        model=model,
        messages=memory_,
        temperature=temperature
        )
    response_ = raw_response_["choices"][0]["message"]["content"]
    return response_

# Run inference

In [ ]:
prompt = "Who let the dogs out?" # Type out the user prompt here

In [ ]:
memory += [{"role": "user", "content": prompt}]
response = prompt_LLM(memory)
memory += [{"role": "assistant", "content": response}]

print(pretty_parse(memory))

# Run interactive inference

In [ ]:
stop = False
stop_string = "__STOP__"
while not stop:
    # Get prompt
    prompt = input("User: ")
    
    # Check stop
    if prompt == stop_string:
        stop = True
        continue

    memory += [{"role": "user", "content": prompt}]

    # show `memory`
    clear_output(wait=True)
    print(pretty_parse(memory))

    # prompt LLM
    response = prompt_LLM(memory)
    memory += [{"role": "assistant", "content": response}]

    # show `memory`
    clear_output(wait=True)
    print(pretty_parse(memory))

    

# Save memory to disk

In [ ]:
prompt = "Summarize the topic of our conversation in four words or less."
memory += [{"role": "user", "content": prompt}]
response = prompt_LLM(memory)

file_name = response.translate(str.maketrans('', '', string.punctuation))
file_name = file_name.replace(' ', '_')
file_name = out_dir_name + "/" + file_name
os.makedirs(out_dir_name, exist_ok=True)
with open(f"{file_name}.p", "wb") as fp:
    pickle.dump(memory[:-1], fp)

# Load memory from disk

In [ ]:
file_name = "" # Specify the file name

In [ ]:
file_name = out_dir_name + "/" + file_name
with open(f"{file_name}.p", "rb") as fp:
    memory = pickle.load(fp)